<a href="https://colab.research.google.com/github/kozyreviva/Climat_DS/blob/main/Air_Indicators/climat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install dataprep



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
# import libraris
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from dataprep.clean import clean_country

In [39]:
# read datasets
df_n2o = pd.read_csv("N2O_Emissions.csv")
df_co2 = pd.read_csv("CO2_Emissions.csv")
df_so2 = pd.read_csv("SO2_emissions.csv")

In [40]:
df_n2o.head()

,Country ID,Country,"Time series - Total N2O emissions, in 1000 tonnes of CO2 equivalent",Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,"N2O emissions, latest year",% change since 1990,"N2O emissions \nper capita, \nlatest year"
0,NaN,NaN,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,Latest Year,1000 tonnes of \nCO2 equivalent,%,kg
1,4.0,Afghanistan,...,...,...,...,...,...,...,...,...,"19,610.00",...,...,...,...,...,2013,"19,610.00",...,0.608
2,8.0,Albania,96.10,136.40,108.50,96.10,96.10,93.00,83.70,77.50,...,...,...,...,...,...,...,2009,120.96,25.87,0.041
3,12.0,Algeria,...,...,...,...,"9,300.00",...,...,...,...,...,...,...,...,...,...,2000,"6,500.70",...,0.209
4,24.0,Angola,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,2005,"13,869.40",...,0.714


In [41]:
def new_col_name(data):
   colnames = list(data.iloc[0])
   colnames[0] = 'country_id'
   colnames[1] = 'country'
   colnames[-3:] = data.columns[-3:]
   data.columns = colnames
   return  data.drop(0)


In [42]:
df_n2o = new_col_name(df_n2o)

In [43]:
df_n2o.head(10)

,country_id,country,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,Latest Year,"N2O emissions, latest year",% change since 1990,"N2O emissions \nper capita, \nlatest year"
1,4.0,Afghanistan,...,...,...,...,...,...,...,...,...,"19,610.00",...,...,...,...,...,2013,"19,610.00",...,0.608
2,8.0,Albania,96.10,136.40,108.50,96.10,96.10,93.00,83.70,77.50,...,...,...,...,...,...,...,2009,120.96,25.87,0.041
3,12.0,Algeria,...,...,...,...,"9,300.00",...,...,...,...,...,...,...,...,...,...,2000,"6,500.70",...,0.209
4,24.0,Angola,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,2005,"13,869.40",...,0.714
5,28.0,Antigua and Barbuda,1.58,...,...,...,...,...,...,...,...,...,...,...,...,...,...,2000,83.70,"5,194.12",1.101
6,32.0,Argentina,"51,823.68",...,...,...,"56,689.61",...,...,"62,089.62",...,...,...,...,...,...,...,2012,"73,154.41",41.16,1.752
7,51.0,Armenia,169.57,...,...,...,...,...,...,...,...,...,...,...,...,...,...,2010,484.07,185.47,0.168
8,36.0,Australia,53.48,52.21,51.57,53.14,54.75,52.30,56.55,59.22,...,65.95,67.44,65.63,65.66,71.36,67.50,2018,67.50,26.20,0.003
9,40.0,Austria,14.50,14.61,13.98,14.08,13.92,14.24,14.34,14.43,...,11.53,11.81,11.84,12.15,11.95,11.83,2018,11.83,-18.39,0.001
10,31.0,Azerbaijan,"3,772.70","1,925.10","2,132.80","1,701.90","1,429.10","1,295.80","1,308.20","1,515.90",...,"2,767.12",...,...,...,...,...,2013,"2,767.12",-26.65,0.295


In [44]:
def replace_to_na(data):
    return data.replace('...', np.NaN)

In [45]:
df_n2o = replace_to_na(df_n2o)

In [46]:
df_n2o.head()

,country_id,country,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,Latest Year,"N2O emissions, latest year",% change since 1990,"N2O emissions \nper capita, \nlatest year"
1,4.0,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"19,610.00",NaN,NaN,NaN,NaN,NaN,2013,"19,610.00",NaN,0.608
2,8.0,Albania,96.10,136.40,108.50,96.10,96.10,93.00,83.70,77.50,...,NaN,NaN,NaN,NaN,NaN,NaN,2009,120.96,25.87,0.041
3,12.0,Algeria,NaN,NaN,NaN,NaN,"9,300.00",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2000,"6,500.70",NaN,0.209
4,24.0,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2005,"13,869.40",NaN,0.714
5,28.0,Antigua and Barbuda,1.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2000,83.70,"5,194.12",1.101


In [47]:
df_n2o_dropna_percent = df_n2o.dropna(subset = "% change since 1990").reset_index(drop= True).copy()
df_n2o_dropna_percent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 35 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   country_id                               85 non-null     float64
 1   country                                  85 non-null     object 
 2   1990                                     85 non-null     object 
 3   1991                                     61 non-null     object 
 4   1992                                     62 non-null     object 
 5   1993                                     62 non-null     object 
 6   1994                                     76 non-null     object 
 7   1995                                     60 non-null     object 
 8   1996                                     62 non-null     object 
 9   1997                                     62 non-null     object 
 10  1998                                     64 non-null

In [48]:
df_n2o_dropna_percent.iloc[:,2:31] = df_n2o_dropna_percent.iloc[:,2:31].replace(to_replace = ',', value = '', regex = True)
df_n2o_dropna_percent.iloc[:,2:31] = df_n2o_dropna_percent.iloc[:,2:31].astype(float)

<ipython-input-48-6cf00a38800a>:2: DeprecationWarning:

In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`



In [49]:
df_n2o_dropna_percent["Latest Year"] = df_n2o_dropna_percent['Latest Year'].astype(int)
df_n2o_dropna_percent["% change since 1990"] = df_n2o_dropna_percent['% change since 1990'].replace(to_replace = ',', value = '', regex = True)
df_n2o_dropna_percent["% change since 1990"] = df_n2o_dropna_percent['% change since 1990'].astype(float)
df_n2o_dropna_percent["N2O emissions, latest year"] = df_n2o_dropna_percent["N2O emissions, latest year"].replace(to_replace = ',', value = '', regex = True)
df_n2o_dropna_percent["N2O emissions, latest year"] = df_n2o_dropna_percent['N2O emissions, latest year'].astype(float)
df_n2o_dropna_percent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 35 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   country_id                               85 non-null     float64
 1   country                                  85 non-null     object 
 2   1990                                     85 non-null     float64
 3   1991                                     61 non-null     float64
 4   1992                                     62 non-null     float64
 5   1993                                     62 non-null     float64
 6   1994                                     76 non-null     float64
 7   1995                                     60 non-null     float64
 8   1996                                     62 non-null     float64
 9   1997                                     62 non-null     float64
 10  1998                                     64 non-null

In [50]:
df_n2o_dropna_percent.head(10)

,country_id,country,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,Latest Year,"N2O emissions, latest year",% change since 1990,"N2O emissions \nper capita, \nlatest year"
0,8.0,Albania,96.10,136.40,108.50,96.10,96.10,93.00,83.70,77.50,...,NaN,NaN,NaN,NaN,NaN,NaN,2009,120.96,25.87,0.041
1,28.0,Antigua and Barbuda,1.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2000,83.70,5194.12,1.101
2,32.0,Argentina,51823.68,NaN,NaN,NaN,56689.61,NaN,NaN,62089.62,...,NaN,NaN,NaN,NaN,NaN,NaN,2012,73154.41,41.16,1.752
3,51.0,Armenia,169.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2010,484.07,185.47,0.168
4,36.0,Australia,53.48,52.21,51.57,53.14,54.75,52.30,56.55,59.22,...,65.95,67.44,65.63,65.66,71.36,67.50,2018,67.50,26.20,0.003
5,40.0,Austria,14.50,14.61,13.98,14.08,13.92,14.24,14.34,14.43,...,11.53,11.81,11.84,12.15,11.95,11.83,2018,11.83,-18.39,0.001
6,31.0,Azerbaijan,3772.70,1925.10,2132.80,1701.90,1429.10,1295.80,1308.20,1515.90,...,2767.12,NaN,NaN,NaN,NaN,NaN,2013,2767.12,-26.65,0.295
7,112.0,Belarus,55.77,54.20,48.47,48.96,41.79,38.96,41.68,42.99,...,48.21,47.31,46.35,46.89,48.44,47.05,2018,47.05,-15.64,0.005
8,56.0,Belgium,33.80,33.36,32.43,33.35,34.71,36.33,37.80,37.05,...,20.63,20.74,20.25,19.34,20.06,19.14,2018,19.14,-43.39,0.002
9,68.0,Bolivia (Plurinational State of),576.60,NaN,NaN,NaN,858.70,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2004,1386.91,140.53,0.153


In [51]:
df_n2o_country_percent = (df_n2o_dropna_percent[['country','% change since 1990']].query("`% change since 1990` >= 100")
                                                                                  .sort_values('% change since 1990',ascending = False)).reset_index(drop = True).copy()
df_n2o_country_alpha = clean_country(df_n2o_country_percent, "country", output_format = 'alpha-3')
df_n2o_country_alpha = df_n2o_country_alpha.rename(columns = {'country_clean':'iso_alpha'})
df_n2o_country_alpha.head()



/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:6751: FutureWarning:

Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.



Country Cleaning Report:
	13 values cleaned (100.0%)
Result contains 13 (100.0%) values in the correct format and 0 null values (0.0%)


,country,% change since 1990,iso_alpha
0,Lao People's Democratic Republic,6625.00,LAO
1,Antigua and Barbuda,5194.12,ATG
2,Mongolia,1380.00,MNG
3,Costa Rica,1302.51,CRI
4,Ethiopia,678.19,ETH


In [52]:
fig = px.bar(df_n2o_country_percent, x = "country", y = "% change since 1990")
fig.show()

In [53]:
df = px.data.gapminder()
fig = px.scatter_geo(df_n2o_country_alpha,
                     locations = "iso_alpha",
                     size = "% change since 1990",
                     projection="natural earth",
                     color= "country")
fig.show()

In [54]:
lst =[]
for i, row in df_n2o_dropna_percent.iloc[:,2:31].items():
    for item in row:
        lst.append([i,item])
df = pd.DataFrame(lst)
df = df.rename(columns = {0:'year',1:'values'})
df

,year,values
0,1990,96.10
1,1990,1.58
2,1990,51823.68
3,1990,169.57
4,1990,53.48
...,...,...
2460,2018,64.46
2461,2018,NaN
2462,2018,1458.15
2463,2018,NaN


In [55]:
new_n2o_country = df_n2o_dropna_percent['country'].copy()
for i in range(28):
   new_n2o_country = pd.concat([new_n2o_country,df_n2o_dropna_percent['country']], ignore_index = True)

new_df = df.join(new_n2o_country)
new_df = clean_country(new_df, "country", output_format = 'alpha-3')
new_df = new_df.rename(columns = {'country_clean':'iso_alpha'})
new_df

/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:6751: FutureWarning:

Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.

                                     

Country Cleaning Report:
	2465 values cleaned (100.0%)
Result contains 2465 (100.0%) values in the correct format and 0 null values (0.0%)


,year,values,country,iso_alpha
0,1990,96.10,Albania,ALB
1,1990,1.58,Antigua and Barbuda,ATG
2,1990,51823.68,Argentina,ARG
3,1990,169.57,Armenia,ARM
4,1990,53.48,Australia,AUS
...,...,...,...,...
2460,2018,64.46,United Kingdom of Great Britain and Northern I...,GBR
2461,2018,NaN,United Republic of Tanzania,TZA
2462,2018,1458.15,United States of America,USA
2463,2018,NaN,Uruguay,URY


In [56]:
# Creating the visualization
fig = px.choropleth(new_df,
                    locations="iso_alpha",
                    color="values",
                    hover_name="country",
                    animation_frame="year"
                   )
fig.update_layout(
    title_text = 'Animated_map',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))